<a href="https://colab.research.google.com/github/viswakimi/News-Article-Categorization/blob/main/newsarticle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.8 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from transformers import DistilBertTokenizer
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

In [18]:
from sklearn.metrics import accuracy_score, classification_report

In [4]:
# Load Dataset
dataset = load_dataset("ag_news")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.07k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [5]:

# Extract texts and labels
texts = [item["text"] for item in dataset["train"]]
labels = [item["label"] for item in dataset["train"]]

In [6]:
# Stratified Sampling
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_idx, test_idx in splitter.split(texts, labels):
    small_train_texts = [texts[i] for i in train_idx[:10000]]
    small_train_labels = [labels[i] for i in train_idx[:10000]]
    small_test_texts = [texts[i] for i in test_idx[:5000]]
    small_test_labels = [labels[i] for i in test_idx[:5000]]


In [7]:

# Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(texts, tokenizer, max_length=512):
    tokens = tokenizer(texts, padding="max_length", truncation=True, max_length=max_length, return_tensors="pt")
    return tokens["input_ids"], tokens["attention_mask"]

X_train_tokens, X_train_masks = tokenize(small_train_texts, tokenizer)
X_test_tokens, X_test_masks = tokenize(small_test_texts, tokenizer)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [8]:
# Encode labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(small_train_labels)
y_test = label_encoder.transform(small_test_labels)


In [9]:
# Convert to PyTorch Dataset
class NewsDataset(Dataset):
    def __init__(self, tokens, masks, labels):
        self.tokens = tokens
        self.masks = masks
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.tokens[idx], self.masks[idx], self.labels[idx]

train_dataset = NewsDataset(X_train_tokens, X_train_masks, y_train)
test_dataset = NewsDataset(X_test_tokens, X_test_masks, y_test)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [10]:
# Define GRU Model
class GRUClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim=128, hidden_dim=256, num_classes=4):
        super(GRUClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        embedded = self.embedding(x)
        gru_out, _ = self.gru(embedded)
        out = self.fc(gru_out[:, -1, :])
        return self.softmax(out)


In [11]:
# Model Initialization
vocab_size = tokenizer.vocab_size
model = GRUClassifier(vocab_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GRUClassifier(
  (embedding): Embedding(30522, 128)
  (gru): GRU(128, 256, batch_first=True)
  (fc): Linear(in_features=256, out_features=4, bias=True)
  (softmax): LogSoftmax(dim=1)
)

In [12]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [13]:
# Training Loop with Model Saving
def train_model(model, train_loader, criterion, optimizer, epochs=2):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for tokens, masks, labels in train_loader:
            tokens, labels = tokens.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(tokens)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")
        torch.save(model.state_dict(), f"gru_news_classifier_epoch{epoch+1}.pth")
        print(f"Model saved for epoch {epoch+1}")

train_model(model, train_loader, criterion, optimizer)

# Final Save
torch.save(model.state_dict(), "gru_news_classifier_final.pth")
print("GRU model trained on stratified sample data and saved successfully!")


Epoch 1, Loss: 1.4008791323661804
Model saved for epoch 1
Epoch 2, Loss: 1.3936605486869813
Model saved for epoch 2
GRU model trained on stratified sample data and saved successfully!


In [19]:
# Define label mapping
label_mapping = {0: "World", 1: "Sports", 2: "Business", 3: "Science/Technology"}

In [21]:
# Function for Predicting Labels
def predict_label(model, tokenizer, text):
    model.eval()
    tokens, masks = tokenize([text], tokenizer)
    tokens = tokens.to(device)
    with torch.no_grad():
        output = model(tokens)
        pred_label = torch.argmax(output, dim=1).cpu().item()
    class_name = label_mapping[pred_label]
    return class_name

# Example usage
input_text = "Stock market is experiencing a significant rise today."
predicted_label = predict_label(model, tokenizer, input_text)
print(f'Predicted Class: {predicted_label}')

Predicted Class: Business
